In [4]:
!pip install torch torchaudio --quiet
!pip install transformers==4.51.0 peft==0.15.0 accelerate datasets --quiet




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.8/410.8 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 85.1 MB/s eta 0:00:00


In [5]:
!unzip turbo_adapter.zip
!unzip turbo_whisper.zip

Archive:  turbo_adapter.zip
  inflating: turbo_adapter/adapter_model.safetensors  
  inflating: turbo_adapter/README.md  
  inflating: turbo_adapter/adapter_config.json  
Archive:  turbo_whisper.zip
  inflating: turbo_whisper/normalizer.json  
  inflating: turbo_whisper/tokenizer.json  
  inflating: turbo_whisper/tokenizer_config.json  
  inflating: turbo_whisper/special_tokens_map.json  
  inflating: turbo_whisper/merges.txt  
  inflating: turbo_whisper/preprocessor_config.json  
  inflating: turbo_whisper/added_tokens.json  
  inflating: turbo_whisper/vocab.json  


In [1]:
import torch, torchaudio
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from peft import PeftModel

ADAPTER_PATH = "turbo_adapter"
PROCESSOR_PATH = "turbo_whisper"
BASE_MODEL = "openai/whisper-large-v3-turbo"

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on:", device)

processor = WhisperProcessor.from_pretrained(PROCESSOR_PATH)
base_model = WhisperForConditionalGeneration.from_pretrained(BASE_MODEL)
model = PeftModel.from_pretrained(base_model, ADAPTER_PATH).to(device)
model.eval()

def transcribe_audio(path):
    waveform, sr = torchaudio.load(path)
    if sr != 16000:
        waveform = torchaudio.functional.resample(waveform, sr, 16000)
    # mono
    audio = waveform.squeeze().numpy()
    inputs = processor.feature_extractor(audio, sampling_rate=16000, return_tensors="pt")
    input_features = inputs.input_features.to(device)  # shape: (1, seq_len, feature_dim)

    with torch.no_grad():
        try:
            # Preferred: pass by keyword
            ids = model.generate(input_features=input_features, max_new_tokens=444) # Reduced max_new_tokens
        except TypeError:
            # Fallback: call the underlying base_model generate
            ids = model.base_model.generate(input_features=input_features, max_new_tokens=444) # Reduced max_new_tokens

    return processor.batch_decode(ids, skip_special_tokens=True)[0]

print("Transcribing tyy.wav ...")
print("TRANSCRIPTION:", transcribe_audio("/content/WhatsApp Audio 2025-10-04 at 14.36.23_66aeb4f8.wav"))

Running on: cpu


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcod

Transcribing tyy.wav ...


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


TRANSCRIPTION:  One of the curds rice is in Pongal Nala


In [11]:
!pip install -U peft



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.9/504.9 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: peft
    Found existing installation: peft 0.15.0
    Uninstalling peft-0.15.0:
      Successfully uninstalled peft-0.15.0
